##Library & Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
pip install tensorflow.io

In [ ]:
from IPython.display import Audio

import tensorflow as tf
import tensorflow_io as tfio
import matplotlib.pyplot as plt
import numpy as np
import glob

In [ ]:
import os
import librosa
import pickle
import pandas as pd
import seaborn as sns

from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from keras.layers import Conv2D, MaxPool2D, Flatten, Dense, Dropout
from keras.models import Sequential
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
#from python_speech_features import mfcc

# **Load Data Gumaman**

In [ ]:
daftar_gdangdut = glob.glob('/content/drive/MyDrive/Dataset/Dangdut/gumam/*')
#daftar_gpop = glob.glob('/content/drive/MyDrive/Dataset01/Pop/Gumam/*')
#daftar_gjpop = glob.glob('/content/drive/MyDrive/Dataset01/JPop/Gumam/*')
daftar_gkpop = glob.glob('/content/drive/MyDrive/Dataset/KPOP/Gumam/*')

In [ ]:
#baca file dan masukin ke list
gumaman_dangdut =[]
for i in range(len(daftar_gdangdut)):
  gumam = tf.io.read_file(daftar_gdangdut[i])
  suara = tfio.audio.decode_mp3(gumam)
  gumaman_dangdut.append(suara)
'''
gumaman_pop =[]
for i in range(len(daftar_gpop)):
  gumam = tf.io.read_file(daftar_gpop[i])
  suara = tfio.audio.decode_mp3(gumam)
  gumaman_pop.append(suara)

gumaman_jpop =[]
for i in range(len(daftar_gjpop)):
  gumam = tf.io.read_file(daftar_gjpop[i])
  suara = tfio.audio.decode_mp3(gumam)
  gumaman_jpop.append(suara)
'''
gumaman_kpop =[]
for i in range(len(daftar_gkpop)):
  gumam = tf.io.read_file(daftar_gkpop[i])
  suara = tfio.audio.decode_mp3(gumam)
  gumaman_kpop.append(suara)

In [ ]:
#train
x=[]
y=[]

#mencacah/motong lagu
for j in range(len(daftar_gdangdut)):
  for i in range(0, len(gumaman_dangdut[j])-44100, 11025):
    x.append(gumaman_dangdut[j][i:i+44100])
    y.append(0)

for j in range(len(daftar_gkpop)):
  for i in range(0, len(gumaman_kpop[j])-44100, 11025):
    x.append(gumaman_kpop[j][i:i+44100])
    y.append(1)

'''
for j in range(len(daftar_gjpop)):
  for i in range(0, len(gumaman_jpop[j])-44100, 11025):
    x.append(gumaman_jpop[j][i:i+44100])
    y.append(2)

for j in range(len(daftar_gpop)):
  for i in range(0, len(gumaman_pop[j])-44100, 11025):
    x.append(gumaman_pop[j][i:i+44100])
    y.append(3)
'''

'\nfor j in range(len(daftar_gjpop)):\n  for i in range(0, len(gumaman_jpop[j])-44100, 11025):\n    x.append(gumaman_jpop[j][i:i+44100])\n    y.append(2)\n\nfor j in range(len(daftar_gpop)):\n  for i in range(0, len(gumaman_pop[j])-44100, 11025):\n    x.append(gumaman_pop[j][i:i+44100])\n    y.append(3)\n'

In [ ]:
x = np.asarray(x)
y = np.asarray(y)

In [ ]:
label_encoder = LabelEncoder()
label_encoded = label_encoder.fit_transform(y)

In [ ]:
label_encoded=label_encoded[:,np.newaxis]

In [ ]:
one_hot_encoder = OneHotEncoder(sparse=False)
one_hot_encoded = one_hot_encoder.fit_transform(label_encoded)

In [ ]:
y = one_hot_encoded

In [ ]:
maks = np.max(x)
mins = np.min(x)

#normalisasi
x = (x-mins)/(maks-mins)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

In [ ]:
y_test = one_hot_encoder.inverse_transform(y_test)

In [ ]:
print(x.shape)
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(9345, 44100, 1)
(7476, 44100, 1)
(7476, 2)
(1869, 44100, 1)
(1869, 1)


# **Model**

In [ ]:
# CNN 1D
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Activation

#size_ = 8 #inputnya ada 8, dr pclass-embarked

model = Sequential()

model.add(Conv1D(128, kernel_size=3, input_shape = (x.shape[1],1), activation = 'relu'))
model.add(Flatten())

model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(64, activation='relu'))

#model.add(Dense(units = 8, activation = 'relu'))
model.add(Dense(units = 2, activation = 'softmax'))

model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(x_train,y_train, batch_size=20, epochs=50)

Epoch 1/50
374/374 [==============================] - 48s 126ms/step - loss: 0.7538 - accuracy: 0.6409
Epoch 2/50
374/374 [==============================] - 47s 126ms/step - loss: 0.5034 - accuracy: 0.7846
Epoch 3/50
374/374 [==============================] - 47s 125ms/step - loss: 0.4891 - accuracy: 0.7974
Epoch 4/50
374/374 [==============================] - 47s 126ms/step - loss: 0.4874 - accuracy: 0.7913
Epoch 5/50
374/374 [==============================] - 47s 125ms/step - loss: 0.4639 - accuracy: 0.8050
Epoch 6/50
374/374 [==============================] - 47s 126ms/step - loss: 0.4439 - accuracy: 0.8151
Epoch 7/50
374/374 [==============================] - 47s 126ms/step - loss: 0.3974 - accuracy: 0.8390
Epoch 8/50
374/374 [==============================] - 47s 126ms/step - loss: 0.3663 - accuracy: 0.8555
Epoch 9/50
374/374 [==============================] - 47s 126ms/step - loss: 0.3322 - accuracy: 0.8713
Epoch 10/50
374/374 [==============================] - 47s 125ms/step - l

In [ ]:
#model.save_weights('f1_classifiergumam.h5')
#read file
model.load_weights('/content/drive/MyDrive/Dataset/f1_classifiergumam.h5')

In [ ]:
lur = model.predict(x_test)

In [ ]:
lur

array([[2.9304340e-03, 9.9706954e-01],
       [7.7997088e-01, 2.2002910e-01],
       [8.3341956e-01, 1.6658041e-01],
       ...,
       [6.2037691e-02, 9.3796235e-01],
       [7.7124387e-01, 2.2875616e-01],
       [8.7676576e-12, 1.0000000e+00]], dtype=float32)

In [ ]:
lur[50]

array([0.99657047, 0.00342958], dtype=float32)

In [ ]:
predictions = np.argmax(lur, axis=1)

In [ ]:
predictions

array([1, 0, 0, ..., 1, 0, 1])

In [ ]:
predict = pd.DataFrame(predictions)
predict

,0
0,1
1,0
2,0
3,0
4,0
...,...
1864,0
1865,1
1866,1
1867,0


In [ ]:
predict[(predict[0]==1)].shape

(843, 1)

In [ ]:
y_test

array([[1],
       [0],
       [0],
       ...,
       [1],
       [1],
       [1]])

In [ ]:
accuracy_score(predictions, y_test)

0.8945960406634564

##Predict with new file

In [ ]:
ftest = '/content/drive/MyDrive/Dataset/Dangdut/gumam/Salinan Dadido - Aca Aca Nehi Nehi.mp3'
ftest

'/content/drive/MyDrive/Dataset/Dangdut/gumam/Salinan Dadido - Aca Aca Nehi Nehi.mp3'

In [ ]:
f_Test = tf.io.read_file(ftest)
s_Test = tfio.audio.decode_mp3(f_Test)
s_Test

<tf.Tensor: shape=(11361496, 1), dtype=float32, numpy=
array([[-1.08603235e-05],
       [-1.64603007e-05],
       [-1.81411378e-05],
       ...,
       [ 3.20703693e-04],
       [ 5.58009138e-04],
       [ 4.87105513e-04]], dtype=float32)>

In [ ]:
input = []
for i in range(0, len(s_Test)-44100, 11025):
    input.append(s_Test[i:i+44100])

input = np.asarray(input)
maks = np.max(input)
min = np.min(input)

input = (input-min)/(maks-min)

In [ ]:
input

array([[[0.4887471 ],
        [0.48873165],
        [0.48872706],
        ...,
        [0.48889175],
        [0.48887008],
        [0.4888478 ]],

       [[0.4889986 ],
        [0.48898825],
        [0.48897216],
        ...,
        [0.48910192],
        [0.4891436 ],
        [0.48912564]],

       [[0.48919842],
        [0.48923653],
        [0.4892743 ],
        ...,
        [0.4889163 ],
        [0.4889482 ],
        [0.4889528 ]],

       ...,

       [[0.49061498],
        [0.49053007],
        [0.49071553],
        ...,
        [0.48118585],
        [0.48203975],
        [0.4828669 ]],

       [[0.48393956],
        [0.48407656],
        [0.48445177],
        ...,
        [0.49234954],
        [0.49252805],
        [0.4926052 ]],

       [[0.49425253],
        [0.49412718],
        [0.4939522 ],
        ...,
        [0.48901045],
        [0.48871914],
        [0.4883934 ]]], dtype=float32)

In [ ]:
input.shape

(1027, 44100, 1)

In [ ]:
len(input)

1027

In [ ]:
hasil = model.predict(input)
hasil = np.argmax(hasil,axis=1)
hasil

array([0, 0, 0, ..., 0, 0, 0])

In [ ]:
hasil = pd.DataFrame(hasil)
jmlh_0 = hasil[(hasil[0]==0)].shape[0]
jmlh_1 = hasil[(hasil[0]==1)].shape[0]

'''
jmlh_1 = 9
jmlh_0 = 22'''

if jmlh_0 > jmlh_1 :
  print('dangdut')
elif jmlh_0 < jmlh_1 :
  print('kpop')
else :
  print('not known')

kpop


In [ ]:
print(jmlh_0)
print(jmlh_1)

510
517


##Flask

In [ ]:
!pip install flask

In [ ]:
!pip install flask-ngrok

In [ ]:
!curl -s https://ngrok-agent.s3.amazonaws.com/ngrok.asc | sudo tee /etc/apt/trusted.gpg.d/ngrok.asc >/dev/null && echo "deb https://ngrok-agent.s3.amazonaws.com buster main" | sudo tee /etc/apt/sources.list.d/ngrok.list && sudo apt update && sudo apt install ngrok   
              

deb https://ngrok-agent.s3.amazonaws.com buster main
Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 https://ngrok-agent.s3.amazonaws.com buster InRelease
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:9 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:13 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:14 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease


In [ ]:
! ngrok authtoken 23N1IcF96hHTFMCLqhpdFkFEo8J_cs4VCtJuHfxbkXDNzyYT

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [ ]:
!cp -r "/content/drive/MyDrive/Dataset/templates" .
!cp -r "/content/drive/MyDrive/Dataset/static" .

In [ ]:
# flask_ngrok_example.py
from flask import Flask, render_template, request, redirect, url_for
from flask_ngrok import run_with_ngrok

ALLOWED_EXTENSIONS = set(['mp3'])

app = Flask(__name__)
run_with_ngrok(app)  # Start ngrok when app is run

def allowed_file(filename):
    return '.' in filename and \
           filename.rsplit('.', 1)[1] in ALLOWED_EXTENSIONS

@app.route("/")
def hello():
    return render_template("index.html")

@app.route("/fungsi1", methods=['POST','GET'])
def fungsi1():
  if request.method == 'POST':
    audio = request.files['audio']
    audio = audio.save('test.mp3')
    f_test = tf.io.read_file('test.mp3')
    s_test = tfio.audio.decode_mp3(f_test)

    input = []
    for i in range(0, len(s_test)-44100, 11025):
        input.append(s_test[i:i+44100])
    input = np.asarray(input)
    maks = np.max(input)
    min = np.min(input)
    input = (input-min)/(maks-min)

    hasil = model.predict(input)
    hasil = np.argmax(hasil,axis=1)
    hasil = pd.DataFrame(hasil)
    jmlh_0 = hasil[(hasil[0]==0)].shape[0]
    jmlh_1 = hasil[(hasil[0]==1)].shape[0]

    if jmlh_0 > jmlh_1 :
      result = 'dangdut'
    elif jmlh_0 < jmlh_1 :
      result = 'kpop'
    else :
      result = 'not known'

    return render_template("fungsi1.html",result="{}".format(result))

  else :
    return render_template("fungsi1.html")

if __name__ == '__main__':
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://fb36-34-125-255-89.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [29/Jan/2022 16:47:00] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [29/Jan/2022 16:47:00] "GET /static/css/base.css HTTP/1.1" 200 -
127.0.0.1 - - [29/Jan/2022 16:47:00] "GET /static/css/bootstrap.css HTTP/1.1" 200 -
127.0.0.1 - - [29/Jan/2022 16:47:00] "GET /static/css/main.css HTTP/1.1" 200 -
127.0.0.1 - - [29/Jan/2022 16:47:00] "GET /static/css/flexslider.css HTTP/1.1" 200 -
127.0.0.1 - - [29/Jan/2022 16:47:00] "GET /static/css/magnific-popup.css HTTP/1.1" 200 -
127.0.0.1 - - [29/Jan/2022 16:47:00] "GET /static/css/fonts.css HTTP/1.1" 200 -
127.0.0.1 - - [29/Jan/2022 16:47:01] "GET /static/img/1.jpg HTTP/1.1" 200 -
127.0.0.1 - - [29/Jan/2022 16:47:01] "GET /static/js/jquery-1.12.4.min.js HTTP/1.1" 200 -
127.0.0.1 - - [29/Jan/2022 16:47:01] "GET /static/js/jquery.flexslider-min.js HTTP/1.1" 200 -
127.0.0.1 - - [29/Jan/2022 16:47:01] "GET /static/js/smooth-scroll.js HTTP/1.1" 200 -
127.0.0.1 - - [29/Jan/2022 16:47:01] "GET /static/js/jquery.magnific-popup.min.js HTTP/1.1" 200 -
